In [77]:
from tkinter import * 
from PIL import Image,ImageTk
import numpy as np
import matplotlib.pyplot as plt
from keras.utils import to_categorical
import os
import cv2

root=Tk()

def train_model():
    

    X=[]#images
    Y=[]#categories


    folder_ped='pedestrian'
    folder_road='road'
    folder_car='car'
    name_encode={'pedestrian':0,'road':1,'car':2}

    def images_to_array(folder,name):
        for image in os.listdir(folder):
            loaded_image=Image.open(os.path.join(folder,image))
            resized_image=Image.Image.resize(loaded_image,[100,100])
            image_array=np.array(resized_image)
            X.append(image_array)
            Y.append(name_encode[name])

            image_flipped=cv2.flip(image_array,1)
            X.append(image_flipped)
            Y.append(name_encode[name])

            image_blurred=cv2.blur(image_array,(2,2))
            X.append(image_blurred)
            Y.append(name_encode[name])

            image_flipped_blurred=cv2.blur(image_flipped,(2,2))
            X.append(image_flipped_blurred)
            Y.append(name_encode[name])

    def show_image(index):
        plt.imshow(X[index])
        plt.show()
        print(Y[index])

    images_to_array(folder_ped,'pedestrian')
    images_to_array(folder_road,'road')
    images_to_array(folder_car,'car')
    Y=to_categorical(Y,num_classes=3)
    X=(np.array(X)-127.5)/127.5
    show_image(50)

    from keras.layers import Activation, Conv2D, Dense, Flatten ,MaxPool2D,BatchNormalization,Dropout
    from keras.models import Sequential

    model=Sequential()
    model.add(Conv2D(32,(4,4),padding='same',activation='relu',input_shape=(100,100,3)))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Conv2D(100,(4,4),padding='same',activation='relu'))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(BatchNormalization())
    model.add(Dropout(rate=0.5))
    model.add(Conv2D(100,(4,4),padding='same',activation='relu'))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(BatchNormalization())
    model.add(Dropout(rate=0.5))


    model.add(Flatten())
    model.add(Dense(124))
    model.add(Activation('relu'))
    model.add(Dense(3))
    model.add(Activation('softmax'))
    model.summary()

    from sklearn.model_selection import train_test_split

    X_train,X_test,y_train,y_test= train_test_split(X,Y,test_size=0.1)
    print(Y[39])

    from keras.optimizers import Adam

    optimizer=Adam(lr=0.001)
    model.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['acc'])
    h=model.fit(X_train,y_train,batch_size=10,epochs=20,validation_data=(X_test,y_test))
    model.save('road_pederstrian.h5')

    plt.plot(h.history['acc'])
    plt.plot(h.history['val_acc'])
    plt.title('CNN accuray')
    plt.xlabel('epoch no')
    plt.ylabel('Accuracy')
    plt.legend(loc='lower right')


    from keras.models import load_model
    model=load_model('road_pederstrian.h5')
    def prediction(index_number):
        img=(np.array(X[index_number])-127.5)/127.5
        img= img.reshape(1,100,100,3)
        prediction=model.predict_classes(img)
        if prediction==0:
            print('prediction: pedestrian')
        elif prediction==1:
            print('prediction: road')
        else:
            print('prediction:car')
        plt.imshow(X[index_number])
        plt.show
        prediction(24)


    
def prediction(image_path):
    
    from PIL import Image
    import numpy as np
    import matplotlib.pyplot as plt
    from keras.utils import to_categorical
    import os
    from keras.models import load_model
    model=load_model('road_pederstrian.h5')
    img_load=Image.open(image_path)
    img_resize=Image.Image.resize(img_load,(100,100))
    img=(np.array(img_resize)-127.5)/127.5
    img= img.reshape(1,100,100,3)
    prediction=model.predict_classes(img)
    if prediction==0:
        
        text1 = Label(root,text="prediction: pedestrian",font=("arial",10,"bold")).place(x=10,y=100)
        print=('prediction: pedestrian')
       
    elif prediction==1:
        text1 = Label(root,text="prediction: road",font=("arial",10,"bold")).place(x=10,y=100)
        print=('prediction: road')
    
       
    else:
        text1 = Label(root,text="prediction: car",font=("arial",10,"bold")).place(x=10,y=100)
        print=('prediction: car')
        
    plt.imshow(img_resize)
    plt.show    

def tk_predict():
    prediction('ped.jpg')
    
    
  
   

title=Label(root,text="Image prediction by Machine learning",pady=20,font=("arial",10,"bold"))
train_btn=Button(root,text="Train",command=train_model,bg='pink',pady=10,padx=10)
predict_btn=Button(root,text="predict",command=tk_predict,bg='pink',pady=10,padx=10) 

#load = Image.open("ped.jpg")
#render =ImageTk.PhotoImage(load)
#img = Label(root, image=render)
#img.image = render
#img.place(x=0, y=0)

title.pack()
train_btn.pack()
predict_btn.pack()
root.geometry('500x500')
root.mainloop()